In [ ]:
<center><h1>2. Vainilla CNNs</h1></center>

In [123]:
# Necessary modules
import pandas as pd
import numpy  as np

from keras.models     import Sequential
from keras.layers     import Dense, Dropout, Activation, Flatten
from keras.layers     import Convolution2D, MaxPooling2D
from keras.optimizers import SGD

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [141]:
# Get the data
data = pd.read_csv(filepath_or_buffer="../data/face_image_project/fold_0_data.txt", sep="\t")
data.head()

,user_id,original_image,face_id,age,gender,x,y,dx,dy,tilt_ang,fiducial_yaw_angle,fiducial_score
0,30601258@N03,10399646885_67c7d20df9_o.jpg,1,"(25, 32)",f,0,414,1086,1383,-115,30,17
1,30601258@N03,10424815813_e94629b1ec_o.jpg,2,"(25, 32)",m,301,105,640,641,0,0,94
2,30601258@N03,10437979845_5985be4b26_o.jpg,1,"(25, 32)",f,2395,876,771,771,175,-30,74
3,30601258@N03,10437979845_5985be4b26_o.jpg,3,"(25, 32)",m,752,1255,484,485,180,0,47
4,30601258@N03,11816644924_075c3d8d59_o.jpg,2,"(25, 32)",m,175,80,769,768,-75,0,34


In [143]:
data = data[["user_id","original_image","face_id","gender"]]
data.head()

,user_id,original_image,face_id,gender
0,30601258@N03,10399646885_67c7d20df9_o.jpg,1,f
1,30601258@N03,10424815813_e94629b1ec_o.jpg,2,m
2,30601258@N03,10437979845_5985be4b26_o.jpg,1,f
3,30601258@N03,10437979845_5985be4b26_o.jpg,3,m
4,30601258@N03,11816644924_075c3d8d59_o.jpg,2,m


In [144]:
data.loc[data.gender != "m", "gender"] = 0
data.loc[data.gender == "m", "gender"] = 1
data.head()

,user_id,original_image,face_id,gender
0,30601258@N03,10399646885_67c7d20df9_o.jpg,1,0
1,30601258@N03,10424815813_e94629b1ec_o.jpg,2,1
2,30601258@N03,10437979845_5985be4b26_o.jpg,1,0
3,30601258@N03,10437979845_5985be4b26_o.jpg,3,1
4,30601258@N03,11816644924_075c3d8d59_o.jpg,2,1


In [145]:
path_template = "../data/face_image_project/aligned/%s/landmark_aligned_face.%d.%s"
data["file_path"] = data[["user_id","face_id","original_image"]].apply(lambda x:  
                                                                   path_template % (x[0],x[1],x[2]),
                                                                   axis=1)
data.head()

,user_id,original_image,face_id,gender,file_path
0,30601258@N03,10399646885_67c7d20df9_o.jpg,1,0,../data/face_image_project/aligned/30601258@N0...
1,30601258@N03,10424815813_e94629b1ec_o.jpg,2,1,../data/face_image_project/aligned/30601258@N0...
2,30601258@N03,10437979845_5985be4b26_o.jpg,1,0,../data/face_image_project/aligned/30601258@N0...
3,30601258@N03,10437979845_5985be4b26_o.jpg,3,1,../data/face_image_project/aligned/30601258@N0...
4,30601258@N03,11816644924_075c3d8d59_o.jpg,2,1,../data/face_image_project/aligned/30601258@N0...


In [153]:
data = data[["file_path","gender"]]
data.head()

,file_path,gender
0,../data/face_image_project/aligned/30601258@N0...,0
1,../data/face_image_project/aligned/30601258@N0...,1
2,../data/face_image_project/aligned/30601258@N0...,0
3,../data/face_image_project/aligned/30601258@N0...,1
4,../data/face_image_project/aligned/30601258@N0...,1


In [154]:
data["gender"].value_counts()

0    2437
1    2047
Name: gender, dtype: int64

Occurences of each gender seem pretty balanced. Looks good to me

In [155]:
Y      = data["gender"]
X_path = data["file_path"]

In [168]:
X_path[0]

'../data/face_image_project/aligned/30601258@N03/landmark_aligned_face.1.10399646885_67c7d20df9_o.jpg'

In [197]:
tempo = X_path[1:3]
tempo.apply(lambda x: img_to_array( load_img(x) ) ) #DOES IT NEED A RESHAPE

1    [[[85.0, 62.0, 70.0], [81.0, 58.0, 66.0], [82....
2    [[[127.0, 108.0, 114.0], [129.0, 110.0, 116.0]...
Name: file_path, dtype: object

In [252]:
def train_generator(data):
    while True:
        start, end = 0, 1
        while end < len(data):
            sample  = data[start:end]
            #print sample["file_path"]

            X = img_to_array( load_img(sample["file_path"][start]) )
            X = X.reshape((1,)+ X.shape)
            Y = sample["gender"][start]
            yield (X, Y)
            start += 1
            end += 1

In [253]:
gen = train_generator(data)
gen.next()

(array([[[[   0.,    0.,    0.],
          [   0.,    0.,    0.],
          [   0.,    0.,    0.],
          ..., 
          [   0.,    0.,    0.],
          [   0.,    0.,    0.],
          [   0.,    0.,    0.]],
 
         [[   0.,    0.,    0.],
          [   0.,    0.,    0.],
          [   0.,    0.,    0.],
          ..., 
          [   0.,    0.,    0.],
          [   0.,    0.,    0.],
          [   0.,    0.,    0.]],
 
         [[   0.,    0.,    0.],
          [   0.,    0.,    0.],
          [   0.,    0.,    0.],
          ..., 
          [   0.,    0.,    0.],
          [   0.,    0.,    0.],
          [   0.,    0.,    0.]],
 
         ..., 
         [[ 160.,  162.,  161.],
          [ 167.,  169.,  168.],
          [ 165.,  165.,  165.],
          ..., 
          [  22.,   15.,    9.],
          [  24.,   17.,   11.],
          [  24.,   17.,    9.]],
 
         [[ 164.,  166.,  165.],
          [ 167.,  169.,  168.],
          [ 167.,  169.,  168.],
          ..., 
  

In [254]:
# Building a vainilla CNN
vainilla_cnn = Sequential()
vainilla_cnn.add(Convolution2D(32, 3, 3, input_shape=(816,816,3)))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(MaxPooling2D(pool_size=(2, 2)))
vainilla_cnn.add(Convolution2D(64, 3, 3))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(Convolution2D(64, 3, 3))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(MaxPooling2D(pool_size=(2, 2)))
vainilla_cnn.add(Flatten())
vainilla_cnn.add(Dense(output_dim=100, input_dim=500))
vainilla_cnn.add(Activation("relu"))
vainilla_cnn.add(Dense(output_dim=200, input_dim=500))
vainilla_cnn.add(Activation("relu"))
vainilla_cnn.add(Dense(output_dim=1))
vainilla_cnn.add(Activation("sigmoid"))

In [255]:
# Compile
vainilla_cnn.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [251]:
# Train
vainilla_cnn.fit_generator(train_generator(data), samples_per_epoch=1, nb_epoch=20)

Epoch 1/20


ValueError: Error when checking model input: expected convolution2d_input_4 to have 4 dimensions, but got array with shape (816, 816, 3)

In [14]:
def rowToImgPath(row):
    userid = row["user_id"]
    faceid = row["face_id"]
    original_img = row["original_image"]
    imgPath = "../data/face_image_project/aligned/%s/landmark_aligned_face.%d.%s" % (userid, faceid, original_img)
    return imgPath

def loadXY(data):
    """ BE INCREDIBLY CAREFUL... The data is large - you could run out of memory and start thrashing.
    It is highly advised to load a small subset at a time (100-200 rows)
    """ 
    X = np.zeros((len(data), 816, 816, 3))
    Y = data["gender"]
    for i in range(len(data)):
        row = data.iloc[i]
        img = load_img(rowToImgPath(row))
        imgArray = img_to_array(img)
        X[i] = imgArray
    return X, Y

def generateXY(data, size=5):
    """ From Keras fit_generator API: The generator is expected to loop over its data indefinitely. """
    while True:
        start, end = 0, size
        while end <= len(data):
            yield loadXY(data[start: end])
            start, end =  start + size, end + size

            
def loadXY2(data):
    """ BE INCREDIBLY CAREFUL... The data is large - you could run out of memory and start thrashing.
    It is highly advised to load a small subset at a time (100-200 rows)
    """ 
    X = np.zeros((len(data), 816, 816, 3))
    Y = data["gender"]
    for i in range(len(data)):
        row = data.iloc[i]
        img = load_img(rowToImgPath(row))
        imgArray = img_to_array(img)
        X[i] = imgArray
    return X, Y

def generateXY2(data, size=32):
    """ From Keras fit_generator API: The generator is expected to loop over its data indefinitely. """
    while True:
        start, end = 0, size
        while end <= len(data):
            yield loadXY(data[start: end])
            start, end =  start + size, end + size
